### Description

Fonctionnement d'un RNN élémentaire (2x1) il prend en entrée deux inputs et sort un output.

Backpropagation : calcul et ajustement des weights à chaque itération. On optimise pour réduire l'écart entre les valeurs prédites et les valeurs réelles.  

Ce fichier reproduit l'algo de backpropagation sur un RNN simple dont  l'architecture est la suivante : 
hidden layer : 2x2
output layer : 2x1

In [40]:
import tensorflow as tf
print("tf version is ", tf.__version__)

tf.keras.backend.set_floatx('float64') # on travaille avec des flottants uniquement

tf version is  2.8.0


### construction du modèle

In [31]:
class model(tf.keras.Model) :
    
    def __init__(self, x, y) :
        super(model, self).__init__()
        
        # input layer : deux inputs constants
        self.x_input = tf.constant(x, dtype = tf.float64)
        self.y_input = tf.constant(y, dtype = tf.float64) 
        
        # layer 1 - hidden layer
        self.w1 = tf.Variable(tf.random.uniform([2,2], 
                                                minval = 0., 
                                                maxval = 1., 
                                                dtype = tf.float64, 
                                                trainable = True,
                                               name = 'w1')) # matrice des poids : 2 neuronnes - 2 poids par neuronne ; trainable = T ie utilisable dans la méthode du gradien descendant
        
        self.b1 = tf.Variable(tf.random.uniform([2], 
                                                minval = 0., 
                                                maxval = 1., 
                                                dtype = tf.float64, 
                                                trainable = True,
                                               name = 'b1')) # vecteur de biais sur les poids 
        
        # layer 2 - output layer
        self.w2 = tf.Variable(tf.random.uniform([2,1], 
                                                minval = 0., 
                                                maxval = 1., 
                                                dtype = tf.float64, 
                                                trainable = True,
                                               name = 'w2')) # matrice des poids : 2 neuronnes - 2 poids par neuronne ; trainable = T ie utilisable dans la méthode du gradien descendant
        
        self.b2 = tf.Variable(tf.random.uniform([1], 
                                                minval = 0., 
                                                maxval = 1., 
                                                dtype = tf.float64, 
                                                trainable = True,
                                               name = 'b2')) # vecteur de biais sur les poids 
    def call(self, inputs) :
        # partie calculs
        inputs = tf.constant(x, dtype = tf.float64)
        
        in_neurons_hidden_layer = tf.add(tf.linalg.matmul(inputs, self.w1), self.b1) # x*w + biais
        out_neurons_hidden_layer = tf.sigmoid(in_neurons_hidden_layer) # on applique une fonction d'activation sigmoide au résultat 

        in_neurons_output_layer = tf.add(tf.linalg.matmul(out_neurons_hidden_layer, self.w2), self.b2) # prend en entrée l'output de la couche hidden précédente + fais le calcul
        out_neurons_output_layer = tf.sigmoid(in_neurons_output_layer) # on applique une fonction d'activation sigmoide au résultat 

        return out_neurons_output_layer

### training

In [23]:
import numpy as np
import pandas as pd

# creation d'un fichier de récap pour tensorboard
current_time = datetime.now().strftime('%Y%m%d-%H%M%S')
logdir = 'logs/gradient_tape/'+ current_time + '/train'
summary_writer = tf.summary.create_file_writer(logdir)

In [33]:
# XOR : illustration du fonctionnement du RNN

x = np.array([[0.,0.], [1.,1.], [0.,1.], [1.,0.]])
y = np.array([1.,0.,1.,1.])

pd.DataFrame({
    'input1' : np.vstack(x).T[0],
    'input2' : np.vstack(x).T[1],
    'output' : y
})

,input1,input2,output
0,0.0,0.0,1.0
1,1.0,1.0,0.0
2,0.0,1.0,1.0
3,1.0,0.0,1.0


In [39]:
# backpropagation phase :: descente du gradient 
# loss function : calcule le delta entre la pred du modele et la valeur attendue et optimise les poids pour réduire le delta
model = model(x, y)
optimizer = tf.optimizers.SGD(learning_rate = 0.01) # optimizer du (stochastic) gradient descent

def loss(outputs_models, targets) :
    
    error = tf.math.subtract(targets, outputs_models)
    return tf.reduce_sum(tf.square(error))


def get_gradient(model, inputs, targets) :
    
    with tf.GradientsTape(watch_accessed_variables = True) as tape : 
        loss_value = loss(model(inputs), targets)
    
    return tape.gradient(loss_value, [model.w1, model.b1, model.w2, model.b2])


def run_network(inputs, targets, epochs) : 
    
    for i in range(epochs) :
        grads = get_gradient(model, inputs, targets)
        optimizer.apply_gradients(zip(grads, [model.w1, model.w2, model.b1, model.b2]))
        loss_epoch = loss(model(inputs), model.y_input)
        
        with summary_writer.as_default() :
            tf.summary_writer('loss', loss_epochs, step=i)
        if i % 100 == 0 :
            print('Loss at the epoch ',{i}, ':', {loss_value})

TypeError: Got an unexpected keyword argument 'trainable'

In [28]:
help(zip)

Help on class zip in module builtins:

class zip(object)
 |  zip(*iterables) --> A zip object yielding tuples until an input is exhausted.
 |  
 |     >>> list(zip('abcdefg', range(3), range(4)))
 |     [('a', 0, 0), ('b', 1, 1), ('c', 2, 2)]
 |  
 |  The zip object yields n-length tuples, where n is the number of iterables
 |  passed as positional arguments to zip().  The i-th element in every tuple
 |  comes from the i-th iterable argument to zip().  This continues until the
 |  shortest argument is exhausted.
 |  
 |  Methods defined here:
 |  
 |  __getattribute__(self, name, /)
 |      Return getattr(self, name).
 |  
 |  __iter__(self, /)
 |      Implement iter(self).
 |  
 |  __next__(self, /)
 |      Implement next(self).
 |  
 |  __reduce__(...)
 |      Return state information for pickling.
 |  
 |  ----------------------------------------------------------------------
 |  Static methods defined here:
 |  
 |  __new__(*args, **kwargs) from builtins.type
 |      Create and ret

In [35]:
# compilation avec la fonction run neetwork
run_network(x, y, epochs = 100) 

NameError: name 'run_network' is not defined

In [ ]:
### tensorboad - evaluation